reference:
- https://www.kaggle.com/code/pelinkeskin/deep-rl-with-stable-baseline3-and-gymnasium-ppo

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import torch as th
from torch import nn
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement
from stable_baselines3.common.monitor import load_results
from stable_baselines3.common.torch_layers import NatureCNN
from stable_baselines3.common.policies import ActorCriticPolicy, ActorCriticCnnPolicy

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [ ]:
LOG_DIR = os.path.join(os.getcwd(), 'log')	# トレーニングのログを保存するディレクトリ
os.makedirs(LOG_DIR, exist_ok=True)

MODEL_DIR = os.path.join(os.pardir, 'models')	# トレーニング済みモデルを保存するディレクトリ
os.makedirs(MODEL_DIR, exist_ok=True)

MODEL_PATH = os.path.join(MODEL_DIR, 'connectx_model')	# トレーニング済みモデルのパス

環境

In [ ]:
# 環境の作成
from environment import ConnectFourGym

training_env = ConnectFourGym(opponent='random')
training_env

In [ ]:
# ログを取得する
training_env = Monitor(training_env, LOG_DIR, allow_early_resets=True)
training_env

In [ ]:
# 「DummyVecEnv」は、OpenAI Gymの環境をベクトル化するための特殊なラッパーです。
# 通常、強化学習アルゴリズムは一度に1つの環境しか処理できませんが、これを使用することで
# 複数の環境を同時に実行することができます。これにより、学習プロセスが効率的になります。
training_env = DummyVecEnv([lambda: training_env])
training_env

In [ ]:
training_env.observation_space.sample()

Training my vector agent with SB3 PPO Algorithm

In [ ]:
#code ref: https://github.com/araffin/rl-baselines-zoo/blob/master/utils/utils.py#L225
def liner_schedule(initial_value: float):
	"""
	Linear learning rate schedule.
	:param initial_value: (float)
	:return: (function)
	"""
	def func(progress_remaining: float) -> float:
		"""
		Progress will decrease from 1 (beginning) to 0
		:param progress_remaining: (float)
		:return: (float)
		"""
		return progress_remaining * initial_value
	return func

In [ ]:
# from agent import CustomCNN

In [ ]:

if os.path.exists(MODEL_PATH):
    print('Loading existing model...')
    agent = PPO.load(MODEL_PATH, env=training_env, verbose=0)
else:
    print('Training new model...')
    agent = PPO(
        policy='MlpPolicy',	# ネットワークアーキテクチャ
        env=training_env,
        n_steps=1536,
        ent_coef=0.001,	# この値が大きいほど、エージェントはさまざまなアクションを試行する傾向があります
        n_epochs=8,
        gae_lambda=0.95,	# Generalized Advantage Estimator。報酬の割引率を制御し、エージェントが将来の報酬にどれだけ価値を置くかを調節
        learning_rate=liner_schedule(3e-4),
        batch_size=512,
        clip_range=0.4,	# PPOのクリップ範囲。PPOは勾配の更新を制限（クリップ）することで、学習の安定性を向上させます
        policy_kwargs={
            # 'features_extractor_class': CustomCNN,	# 特徴抽出器のクラス
            'log_std_init': -2,	# ログスケールでの標準偏差の初期値
            'ortho_init': False,	# 直交初期化の有無
        },
        verbose=1
    )

In [ ]:
print(agent.policy)

In [ ]:
eval_env = ConnectFourGym()
eval_env = Monitor(eval_env, LOG_DIR)
eval_env = DummyVecEnv([lambda: eval_env])

eval_callback = EvalCallback(eval_env,
                             best_model_save_path=LOG_DIR,
                             log_path=LOG_DIR,
                             eval_freq=1000,
                             render=False)

In [ ]:
for key, p in agent.get_parameters()['policy'].items():
    print(key, p.numel())
print(f"Total number of trainable parameters: {sum(p.numel() for ey, p in agent.get_parameters()['policy'].items())}")

In [ ]:
# Train the model for a large number of timesteps
agent.learn(
    total_timesteps=50000,
    reset_num_timesteps=True,
    callback=eval_callback
)